In [2]:
import numpy as np
import pandas as pd
import glob
import os
from pathlib import Path
from itertools import product
import regex as re

In [42]:
DATA = '/Users/caldera/Documents/EPFL/GAP/Project/Projet-Gameplay/wip/kevin/data'
FPS=60

p2 = [''.join(item) for item in product('RrLlJj', repeat=2)]
p3 = [''.join(item) for item in product('RrLlJj', repeat=3)]
p4 = [''.join(item) for item in product('RrLlJj', repeat=4)]

list_patterns = [[], ['R', 'r', 'L', 'l', 'J', 'j'], p2, p3, p4]


In [43]:
def count_overlapping(text, search_for):
    return len(re.findall(search_for, text, overlapped=True))

In [44]:
df = pd.read_csv('/Users/caldera/Documents/EPFL/GAP/Project/Projet-Gameplay/wip/kevin/data/keylogs/ground_truth/2022-11-25-12_13_37.csv')
df_trim = df[(df['KEY'] == 'Key.left') | (df['KEY'] == 'Key.right') | (df['KEY'] == 'Key.space')]

In [45]:
#R -> Down, r -> Up
df_trim.loc[(df_trim['KEY'] == 'Key.right') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'R'
df_trim.loc[(df_trim['KEY'] == 'Key.right') & (df_trim['STATUS'] == 'UP'), 'action'] = 'r'

df_trim.loc[(df_trim['KEY'] == 'Key.left') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'L'
df_trim.loc[(df_trim['KEY'] == 'Key.left') & (df_trim['STATUS'] == 'UP'), 'action'] = 'l'

df_trim.loc[(df_trim['KEY'] == 'Key.space') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'J'
df_trim.loc[(df_trim['KEY'] == 'Key.space') & (df_trim['STATUS'] == 'UP'), 'action'] = 'j'

/var/folders/0y/g4f502v57jd_s8zx0025dl3w0000gn/T/ipykernel_36776/3547076604.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trim.loc[(df_trim['KEY'] == 'Key.right') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'R'


In [46]:
df_trim

,FRAME,KEY,STATUS,action
4,186,Key.right,DOWN,R
6,302,Key.space,DOWN,J
7,311,Key.space,UP,j
8,330,Key.right,UP,r
10,353,Key.right,DOWN,R
...,...,...,...,...
6729,130284,Key.left,UP,l
6730,130295,Key.right,DOWN,R
6731,130323,Key.right,UP,r
6732,130323,Key.space,DOWN,J


In [53]:
#Naive
candidates = dict()
best_timeframe_size = 0

for second_tick in np.linspace(5, 10, 11): #3., 3.1 --> 4.9, 5.
    n_frame = second_tick*FPS

    list_slices = list()
    slice_size = int(df_trim.FRAME.max()/n_frame)

    for period in range(slice_size):
        slice = df_trim[(df_trim.FRAME >= period*n_frame) & (df_trim.FRAME < (period+1)*n_frame)]
        list_slices.append(slice)


    #for i, list_pattern in enumerate(list_patterns[2:]):
    for slice in list_slices:
        str = slice.action.str.cat()

        freq = pd.DataFrame(p4, columns=['pattern'])
        freq['count'] = ''
        freq['frequency'] = ''
        patterns = freq['pattern'].tolist()

        for item in patterns:
            freq.loc[freq['pattern'] == item, 'count'] = count_overlapping(str, item)

        freq = freq[freq['count'] > 0]
        n_pattern = freq['count'].sum()

        for item in patterns:
            freq.loc[freq['pattern'] == item, 'frequency'] = freq['count']/n_pattern * 100

        candidates[second_tick] = freq.sort_values(by=['frequency'], ascending=False).head(3)


In [54]:
candidates

{5.0: Empty DataFrame
 Columns: [pattern, count, frequency]
 Index: [],
 5.5:      pattern count frequency
 175     RJjr     1      25.0
 245     rRJj     1      25.0
 1050    JjrR     1      25.0,
 6.0:     pattern count  frequency
 189    Rjrl     1  11.111111
 312    rLJR     1  11.111111
 581    LJRj     1  11.111111,
 6.5:      pattern count frequency
 312     rLJR     1      20.0
 1052    JjrL     1      20.0
 1073    JjJj     1      20.0,
 7.0:     pattern count frequency
 178    RJjJ     1      50.0
 245    rRJj     1      50.0,
 7.5:      pattern count  frequency
 175     RJjr     2  33.333333
 1050    JjrR     2  33.333333
 245     rRJj     1  16.666667,
 8.0:      pattern count  frequency
 175     RJjr     2  33.333333
 1050    JjrR     2  33.333333
 245     rRJj     1  16.666667,
 8.5:      pattern count  frequency
 175     RJjr     2  22.222222
 1050    JjrR     2  22.222222
 245     rRJj     1  11.111111,
 9.0:      pattern count  frequency
 175     RJjr     2  22.222222


In [49]:
np.linspace(5, 10, 11)

array([ 5. ,  5.5,  6. ,  6.5,  7. ,  7.5,  8. ,  8.5,  9. ,  9.5, 10. ])

In [52]:
list_slices[0]

,FRAME,KEY,STATUS,action
4,186,Key.right,DOWN,R
6,302,Key.space,DOWN,J
7,311,Key.space,UP,j
8,330,Key.right,UP,r
10,353,Key.right,DOWN,R
11,358,Key.right,UP,r
12,361,Key.space,DOWN,J
13,367,Key.right,DOWN,R
14,380,Key.space,UP,j
15,387,Key.right,UP,r


In [57]:

def compute_lps(pattern):
    # Longest Proper Prefix that is suffix array
    lps = [0] * len(pattern)

    prefi = 0
    for i in range(1, len(pattern)):
        
        # Phase 3: roll the prefix pointer back until match or 
        # beginning of pattern is reached
        while prefi and pattern[i] != pattern[prefi]:
            prefi = lps[prefi - 1]

        # Phase 2: if match, record the LSP for the current `i`
        # and move prefix pointer
        if pattern[prefi] == pattern[i]:
            prefi += 1
            lps[i] = prefi

        # Phase 1: is implicit here because of the for loop and 
        # conditions considered above

    return lps



def kmp(pattern, text):
    match_indices = []
    count = 0
    pattern_lps = compute_lps(pattern)

    patterni = 0
    for i, ch in enumerate(text):
        
        # Phase 3: if a mismatch was found, roll back the pattern
        # index using the information in LPS
        while patterni and pattern[patterni] != ch:
            patterni = pattern_lps[patterni - 1]

        # Phase 2: if match
        if pattern[patterni] == ch:
            # If the end of a pattern is reached, record a result
            # and use infromation in LSP array to shift the index
            if patterni == len(pattern) - 1:
                match_indices.append(i - patterni)
                count += 1
                patterni = pattern_lps[patterni]
            
            else:
                # Move the pattern index forward
                patterni += 1

        # Phase 1: is implicit here because of the for loop and 
        # conditions considered above

    return match_indices, count


def remove_impossible_patterns(pattern_list):

    patterns_cleaned = list(pattern_list)

    for sub in list(patterns_cleaned):
        if 'RR' in sub:
            patterns_cleaned.remove(sub)
            continue
        elif 'LL' in sub:
            patterns_cleaned.remove(sub)
            continue
        elif 'JJ' in sub:
            patterns_cleaned.remove(sub)
            continue
        elif 'rr' in sub:
            patterns_cleaned.remove(sub)
            continue
        elif 'll' in sub:
            patterns_cleaned.remove(sub)
            continue
        elif 'jj' in sub:
            patterns_cleaned.remove(sub)
            continue

    return patterns_cleaned

In [75]:
candidates = dict()
best_timeframe_size = 0

p4 = [''.join(item) for item in product('RLJ', 'RrLlJj', 'RrLlJj', 'RrLlJj')]
p4_clean = remove_impossible_patterns(p4)

for second_tick in np.linspace(3, 10, 15): #3., 3.5, 4, ..., 9.5, 10.
    n_frame = second_tick*60
    slice_size = int(df_trim.FRAME.max()/n_frame)


    freq = pd.DataFrame(index=p4_clean, columns=['count', 'frequency'])
    freq['count'] = 0
    freq['frequency'] = 0

    for period in range(slice_size):
        slice = df_trim[(df_trim.FRAME >= period*n_frame) & (df_trim.FRAME < (period+1)*n_frame)]

        str = slice.action.str.cat()

        for item in freq.index:
            freq.loc[item, 'count'] += kmp(item, str)[1]

    n_pattern = freq['count'].sum()

    freq = freq.loc[freq['count'] > 0]

    for item in freq.index:
        if n_pattern > 0:
            freq.loc[item, 'frequency'] = freq.loc[item, 'count']/n_pattern * 100
        else:
            freq.loc[item, 'frequency'] = np.NaN

    candidates[second_tick] = freq.sort_values(by=['count'], ascending=False).head(15)

    print(second_tick, n_pattern)



3.0 1554
3.5 1650
4.0 1761
4.5 1791
5.0 1859
5.5 1914
6.0 1966
6.5 1969
7.0 2000
7.5 2044
8.0 2061
8.5 2079
9.0 2091
9.5 2111
10.0 2123


In [79]:
p4 = [''.join(item) for item in product('RLJ', 'RrLlJj', 'RrLlJj', 'RrLlJj')]
p4_clean = remove_impossible_patterns(p4)

p5 = [''.join(item) for item in product(p4_clean, 'RrLlJj')]

test = [''.join(item) for item in product(freq.index, 'RrLlJj')]
print(len(p4_clean), len(p5), len(freq.index), len(test))

375 2250 117 702


In [3]:
df = pd.read_csv('/Users/caldera/Documents/EPFL/GAP/Project/Projet-Gameplay/wip/kevin/data/keylogs/ground_truth/2022-11-25-12_13_37.csv')
df_trim = df[(df['KEY'] == 'Key.left') | (df['KEY'] == 'Key.right') | (df['KEY'] == 'Key.space')]

#R -> Down, r -> Up
df_trim.loc[(df_trim['KEY'] == 'Key.right') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'R'
df_trim.loc[(df_trim['KEY'] == 'Key.right') & (df_trim['STATUS'] == 'UP'), 'action'] = 'r'

df_trim.loc[(df_trim['KEY'] == 'Key.left') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'L'
df_trim.loc[(df_trim['KEY'] == 'Key.left') & (df_trim['STATUS'] == 'UP'), 'action'] = 'l'

df_trim.loc[(df_trim['KEY'] == 'Key.space') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'J'
df_trim.loc[(df_trim['KEY'] == 'Key.space') & (df_trim['STATUS'] == 'UP'), 'action'] = 'j'
df_trim.action.str.cat()

/var/folders/0y/g4f502v57jd_s8zx0025dl3w0000gn/T/ipykernel_45841/1819871912.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trim.loc[(df_trim['KEY'] == 'Key.right') & (df_trim['STATUS'] == 'DOWN'), 'action'] = 'R'


'RJjrRrJRjrJRrRjrJLljLJljLlRJjJjJjJjJjJLjrJljRJjrLlRJjJjrLJjRlrRrRJjJjJjrLJljRJjrLlRJrjRJjrLlJjRJLrljLJljRrRJjrLlLJjlRLJrjlRrRJjJjJrjJRjJjrRJjrRJjJjJjJjJjrRJjrJjRrRrRrRrRrRrRJjJjJjJjJjJjrRJjJjJjJrjRrRrLlRrRrRrJjJjJjRrRrRrJjJjLlRJjrJRjrJjJjRJjrLlRJjrRJrjRrRJrjLlRJrjRJrjRJjrRrRrRJjrLJljRrJRrjRrLJjlJjRJjJjJjJjLrJjJjJjJjlRrRJjrRJjrRrRJjJjJjJjJjJjJrjLJjJjlRJjrLJljLlRJjrLJRljrRrRJjrJjRJjJjrLlRrLlRJjrLlRrLlRrRrJRjJjrLlJjRrJjLlJjLJljRrJRrjLlRrRrRrJjRrJjLlRrJjRrRrJLljRJjrRrLlRJjJjJjJjJjJjJjJjJjrLJjlRrLlJjJjJjJjJjJjJjLlJRrjRJjrJjJjJjJjRJjrLJjlJjJjJjJjRrRJjrLlLlLlRJjJjJjrLlRJjrRrJjRrRrRrJjLlJjRJrjRrRJjJjrRrRrRrLlJjRrJLjlLJjlRrLJjlRrJRjrLJjlJjLJljLlLlLlLlLlLlLlLlLlLlJjRrJLjlJLljLJjlLJjlJjRrLJljJjLlRrLJljJLRljrLlLJljLlJLjlJLjlLJjlRJjrLJjlRrJRjrLlLJljLJjlJjRrLJjlJLljRrJjRJjJjJjJjJjJjJjrLJjJjJjlRJjrRrLlLlLlRJjJjJjrJRjrRJjrRrRrLlLlRJjLrlRrLJjlRJjrRJjrRrRJjrRJrjRrLJjlRrRJjrLlRrJRjrRrRJjJjrLlRJjrJRjrRrRrRrLlRJjrLJjlRrRrLJjlRJjJjrLlRrLlRJjJjJjrRJjrLJjlRrJLjlRJjrRrJjJLjlRrRJjrRJjrLJjlRrJRjrLlJjRJjrRJrjJjL

In [ ]:
p3 = [''.join(item) for item in product('RLJ', 'RrLlJj', 'RrLlJj')]
p3_clean = remove_impossible_patterns(p3)


list_patterns_set = [[], [l for l in 'RrLlJj'], [''.join(item) for item in product('RLJ', 'RrLlJj')], p3_clean]

columns = pd.MultiIndex.from_product([np.linspace(2, 10, 17), ['count', 'frequency', 'best_freq']], names=["window (s)", "stat"])

freqs = pd.DataFrame(columns=columns)
best_freq_locs = dict()

for i in range(3,11): # on fait pour des patters de longueur 3 à 10
    print('starting size', i)
    freq = pd.DataFrame(0, index=list_patterns_set[i], columns=columns)
    freq.loc[:, pd.IndexSlice[:, 'best_freq']] = False    
    best_freq_loc = dict(zip(freq.index, [[0,0]]*len(freq.index))) # va servir a stocker la fenetre pour laquelle on a la plus haute frequence pour un pattern donné ('frequency','window')

    for pattern in freq.index:
        freq.loc[pattern, (window, 'count')] += kmp(pattern, str)[1] #pour chaque pattern on compte les occurences

    n_pattern = freq[window, 'count'].sum() #nb total de pattern detecté pour une window

    #freq = freq.loc[freq[window, 'count'] > 0]   #on garde que les patterns apparaissants 

    # on calcule et record si necessaire les frequences et la plus haute frequence
    if n_pattern > 0:
        for pattern in freq.index:
            freq.loc[pattern, (window, 'frequency')] = freq.loc[pattern, (window, 'count')]/n_pattern * 100
            
            if freq.loc[pattern, (window, 'frequency')] > best_freq_loc[pattern][0]:
                freq.loc[pattern, (best_freq_loc[pattern][1], 'best_freq')] = False
                freq.loc[pattern, (window, 'best_freq')] = True

                best_freq_loc[pattern][0] = freq.loc[pattern, (window, 'frequency')]
                best_freq_loc[pattern][1] = window


        #candidates[window] = freq.sort_values(by=['count'], ascending=False).head(15)

        print(window, n_pattern)

    print(len(freq))
    #on concat
    freqs = pd.concat([freqs, freq])
    best_freq_locs = best_freq_locs | best_freq_loc

    #on crée le set de patterns de taille k+1 : si un pattern de taille k n'apparait pas dans la string,
    #alors aucun pattern de taille k+1 commencant par ses k caracteres n'existe non plus
    list_patterns_set.append([''.join(pattern) for pattern in product(freq.index, 'RrLlJj')]) 

freqs
